### Import libraries

In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from geopy.geocoders import Nominatim
import geocoder

import time
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import regex as re

### Web Scraping

In [2]:
chromedriver = "D:\Downloads\chromedriver_win32\chromedriver.exe" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)

sys.path.append(chromedriver)
driver = webdriver.Chrome(chromedriver)

In [3]:
zoocasa_url = "https://www.zoocasa.com/search?latitude=43.724004651806176&longitude=-79.39055077202798&zoom=12"
driver.get(zoocasa_url)

signin = driver.find_elements_by_link_text("Sign in")[0]
signin.click()
username = driver.find_element_by_id("text-input-email-730296")
username.clear()
username.send_keys("yhchen518@gmail.com")
password = driver.find_element_by_id("text-input-password-76721")
password.clear()
password.send_keys("danielchen84")
signpw = driver.find_elements_by_tag_name("button")[-1]
signpw.click()

#### Login to zoocasa website in order to access more information of estates

In [4]:
zoocasa_url = "https://www.zoocasa.com"
driver = webdriver.Chrome(chromedriver)
driver.get(zoocasa_url)

uid=""
pwd=""

time.sleep(5)
signin = driver.find_elements_by_link_text("Sign in")[0]
signin.click()

time.sleep(5)
username = driver.find_element_by_id("text-input-email-166647")
username.clear()
username.send_keys(uid)

password = driver.find_element_by_id("text-input-password-76721")
password.clear()
password.send_keys(pwd)

time.sleep(5)
signpw = driver.find_elements_by_tag_name("button")[-1]
signpw.click()

soup = BeautifulSoup(driver.page_source, 'html.parser')
pages = str(soup.find_all("em")[0]).lstrip("<em>").rstrip("</em>")
pages = int(pages[pages.index('f')+2::])

#### Start to do the scraping

In [5]:
house_infos = []

time.sleep(5)
city = driver.find_elements_by_tag_name("input")[0]
city.clear()
city.send_keys("Toronto, ON")

time.sleep(5)
driver.find_elements_by_tag_name("button")[0].click()

time.sleep(5)
driver.find_elements_by_css_selector("path.zoom-minus")[0].click()

time.sleep(5)
driver.find_elements_by_css_selector("path.zoom-minus")[0].click()

time.sleep(15)
soup = BeautifulSoup(driver.page_source, 'html.parser')
pages = str(soup.find_all("em")[0]).lstrip("<em>").rstrip("</em>")
pages = int(pages[pages.index('f')+2::])

time.sleep(5)
for i in range(pages):
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    listing_houses = soup.find_all("div", class_="card-wrapper")
    
    print("It is scriping at " + str(i+1) + "/" + str(pages) + " pages...")
    
    for house in listing_houses:
    
        house_address = house.find_all("div", {"class": "street"})
        house_details = house.find_all("div", {"class": "details"})
        house_price = house.find_all("div", {"class": "price"})

        house_info = [house_address, house_details, house_price]
        house_infos.append(list(map(lambda x: [info.getText() for info in x][0].strip("\n"),house_info)))
    

    
    if i < pages-1:
        breakcount = 0
        time.sleep(5)
        while driver.find_elements_by_css_selector("a.icon-arrow-right-open") == False:
            time.sleep(5)
            breakcount += 1
            if breakcount > 20:
                break
        driver.find_elements_by_css_selector("a.icon-arrow-right-open")[0].click()
      
    time.sleep(20)

It is scriping at  1 /72 pages...
It is scriping at  2 /72 pages...
It is scriping at  3 /72 pages...
It is scriping at  4 /72 pages...
It is scriping at  5 /72 pages...
It is scriping at  6 /72 pages...
It is scriping at  7 /72 pages...
It is scriping at  8 /72 pages...
It is scriping at  9 /72 pages...
It is scriping at  10 /72 pages...
It is scriping at  11 /72 pages...
It is scriping at  12 /72 pages...
It is scriping at  13 /72 pages...
It is scriping at  14 /72 pages...
It is scriping at  15 /72 pages...
It is scriping at  16 /72 pages...
It is scriping at  17 /72 pages...
It is scriping at  18 /72 pages...
It is scriping at  19 /72 pages...
It is scriping at  20 /72 pages...
It is scriping at  21 /72 pages...
It is scriping at  22 /72 pages...
It is scriping at  23 /72 pages...
It is scriping at  24 /72 pages...
It is scriping at  25 /72 pages...
It is scriping at  26 /72 pages...
It is scriping at  27 /72 pages...
It is scriping at  28 /72 pages...
It is scriping at  29 /72 pag

### Form the dataframe of the states around Toronto

In [6]:
df = pd.DataFrame(data = house_infos)
df.columns = ['Address', 'Home Type', 'Price']

#Make price value to be integer
df["Price"] = [str(x[2].replace("$","").replace("\n","").replace(",", "").replace(" ","")) for x in house_infos]

#Make adress to be available searching location on Nominatim 
addr_series = df["Address"].copy()
for idx, address in enumerate(addr_series):
    if "-" in address:
        minusidx = address.index('-')
        addr_series.set_value(idx, addr_series[idx][minusidx+1::])

#### Add postal code to dataframe

In [7]:
#Getting postal code from geocoder.canadapost
postalcodes = []
addr_index = 1
total_addr = df.shape[0]
for address in addr_series.tolist():
    g = geocoder.canadapost(address + ", Toronto")
    postalcodes.append(g.postal[0:3])
    addr_index+=1
df.insert(0, "Postal Code", postalcodes)

In [8]:
df.to_csv("Toronto_Sur_Estate.csv", index=False)

#### Due to a lot of estates are not belonged to Toronto, drop the estates that not related to this project

In [9]:
unique_postal = pd.DataFrame({'Postal Code':df["Postal Code"].unique()})

lat_lng_coords = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng_coords_list = lat_lng_coords['Postal Code'].tolist()
#Delete the postal code not belong to Toronto
postal_notToronto = []
for postalcode in unique_postal['Postal Code']:
    if postalcode not in lat_lng_coords_list:
        unique_postal = unique_postal[unique_postal['Postal Code'] != postalcode]
        postal_notToronto.append(postalcode)
unique_postal = unique_postal.reset_index()

In [10]:
df_Tor_Est = df.copy()
new_addr_series = addr_series.copy()

for postalcode in postal_notToronto:
    df_Tor_Est = df_Tor_Est[df_Tor_Est['Postal Code'] != postalcode]
    new_addr_series = new_addr_series[new_addr]

df_Tor_Est = df_Tor_Est.reset_index()
df_Tor_Est.to_csv("Toronto_Estate.csv", index = False)

#### Use Nominatim to find the coordinat of each estate

In [11]:
geolocator = Nominatim()
latlon = []
new_addr_series = df_Tor_Est["Address"].copy()
for idx, address in enumerate(new_addr_series):
    if "-" in address:
        minusidx = address.index('-')
        new_addr_series.set_value(idx, new_addr_series[idx][minusidx+1::])
addr_list = new_addr_series.tolist()
for address in addr_list:
    location = geolocator.geocode(address + ", Ontario")
    if location != None:
        latlon.append([location.latitude,location.longitude])
    else:
        latlon.append([np.nan, np.nan])
    
lat_lon_df = pd.DataFrame(data = latlon)
lat_lon_df.columns = ["Latitude", "Longitude"]
df_Tor_Est = pd.concat([df_Tor_Est, lat_lon_df], axis=1, sort=False)

#### Due to there are some invalid address that need to be dropped

In [12]:
df_Tor_Est.shape

(1589, 6)


In [13]:
df_Tor_Est = df_Tor_Est[np.invert(np.isnan(df_Tor_Est['Latitude']))]
df_Tor_Est.shape

(1519, 6)


In [14]:
df_Tor_Est.to_csv("Toronto_Estate.csv", index = False)

### Update unique postal codes coordinate for foursquare

In [15]:
postalcodes = []
for address in addr_series:
    g = geocoder.canadapost(address + ", Toronto")
    postalcodes.append(g.postal[0:3])
df_postal = pd.DataFrame({'PostalCode': postalcodes})

In [16]:
updated_unique_postal = pd.DataFrame({'Postal Code':df_Tor_Est["Postal Code"].unique()})

postal_codes_coord = pd.Series(map(lambda x: lat_lng_coords[['Latitude', 'Longitude']][lat_lng_coords['Postal Code'] == x].values.tolist()[0], updated_unique_postal['Postal Code']))
postal_codes_coord_df = pd.DataFrame(columns=['Latitude', 'Longitude'], data = list(postal_codes_coord))
postal_codes_coord_df = pd.concat([updated_unique_postal, postal_codes_coord_df], axis=1, sort=False)

### Use foursquare to access nearby venues
due to there are much of postal code areas, I spent two days to fininsh the dataframe

In [17]:
def getNearbyVenues(names, latitudes, longitudes,CLIENT_ID, CLIENT_SECRET, VERSION,LIMIT, radius):
    
    venues_list=[]
    df_index = 1
    totalnum = names.shape[0]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        df_index+=1    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Foursquare ID and Scecretc
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 83

Toronto_venues_1 = getNearbyVenues(names=postal_codes_coord_df['Postal Code'][0:30],
                                   latitudes=postal_codes_coord_df['Latitude'][0:30],
                                   longitudes=postal_codes_coord_df['Longitude'][0:30],
                                   CLIENT_ID=CLIENT_ID,
                                   CLIENT_SECRET=CLIENT_SECRET,
                                   VERSION=VERSION,
                                   LIMIT=LIMIT,
                                   radius=2000
                                  )

In [19]:
#Foursquare ID and Scecretc
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 86

Toronto_venues_2 = getNearbyVenues(names=postal_codes_coord_df['Postal Code'][30::],
                                   latitudes=postal_codes_coord_df['Latitude'][30::],
                                   longitudes=postal_codes_coord_df['Longitude'][30::],
                                   CLIENT_ID=CLIENT_ID,
                                   CLIENT_SECRET=CLIENT_SECRET,
                                   VERSION=VERSION,
                                   LIMIT=LIMIT,
                                   radius=2000
                                  )

In [20]:
Toronto_venues = pd.concat([Toronto_venues_1, Toronto_venues_2], axis=0, sort=False).reset_index()
Toronto_venues.shape

(4876, 8)


In [21]:
Toronto_venues.to_csv("Toronto_Venues.csv", index = False)